In [47]:
# deep learning libraries (pytorch)
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import datasets, transforms, models

# importing dataset
import os
from PIL import Image
import cv2
import pandas as pd

# data processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# visualization
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import time
# import visualkeras
from ann_visualizer.visualize import ann_viz
from torchviz import make_dot
from torchsummaryX import summary

In [48]:
trainSet = pd.read_csv('Dataset/train.csv')
classes = trainSet[['id','species']].copy()
classes['id'] = classes['id'].astype(str)
classes['label'] = LabelEncoder().fit_transform(classes['species'])

image_folder = 'Dataset/images/'
imgs = []
labels = []
for i in sorted(os.listdir(image_folder)):
    id = i.split('.')[0]
    if id in classes['id'].values:
        labels.append(classes[classes['id'] == id]['label'].values[0])
        image = Image.open(os.path.join(image_folder, i)).convert('1')
        imgs.append(image)

print(f"There are {len(imgs)} images in the dataset")
print(f"There are {len(np.unique(labels))} labels in the dataset")

There are 990 images in the dataset
There are 99 labels in the dataset


In [49]:
X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.2, random_state=42, stratify=labels)
print(f"There are {len(X_train)} images in the training set")
print(f"There are {len(X_test)} images in the test set")
print(f"There are {len(np.unique(y_train))} classes in the training set")
print(f"There are {len(np.unique(y_test))} classes in the test set")

There are 792 images in the training set
There are 198 images in the test set
There are 99 classes in the training set
There are 99 classes in the test set


In [50]:
print(f"Average image size in the training set: {np.mean([np.array(x).shape for x in X_train], axis=0)}")

Average image size in the training set: [491.41919192 688.47727273]


In [51]:
class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.transform(image=np.array(self.images[index],dtype=np.float32))['image']
        label = torch.tensor(self.labels[index], dtype=torch.long)
        return image, label


    
transform = A.Compose([
    A.Resize(32, 32),
    ToTensorV2()
])

trainDataSet = CustomDataSet(images=X_train, labels=y_train, transform=transform)
testDataSet = CustomDataSet(images=X_test, labels=y_test, transform=transform)

In [52]:
class CNNmodel(nn.Module):
    def __init__(self):
        super(CNNmodel, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1) # 32x32x1 -> 32x32x64
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1) # 32x32x64 -> 32x32x64
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # 32x32x64 -> 16x16x64

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1) # 16x16x64 -> 16x16x128
        self.relu3 = nn.ReLU()

        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1) # 16x16x128 -> 16x16x128
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2) # 16x16x128 -> 8x8x128
        
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1) # 8x8x128 -> 8x8x128
        self.relu5 = nn.ReLU()

        self.flatten = nn.Flatten() # 8x8x128 -> 8192
        self.fc1 = nn.Linear(128*8*8, 256) # 8192 -> 256
        self.relu6 = nn.ReLU()
        self.fc2 = nn.Linear(256, 99) # 256 -> 99

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.relu3(self.conv3(x))
        x = self.pool4(self.relu4(self.conv4(x)))
        x = self.relu5(self.conv5(x))

        x = self.fc1(self.flatten(x))
        x = self.fc2(self.relu6(x))
        
        x = F.softmax(x, dim=1)

        return x


In [56]:
m = CNNmodel()
writer = SummaryWriter('runs/CNNmodel'+time.strftime("%Y%m%d-%H%M%S"))
writer.add_graph(m, torch.zeros([1,1,32,32]))
writer.close()

In [57]:
batch_size = 8
trainDataLoader = torch.utils.data.DataLoader(trainDataSet, batch_size=batch_size)
testDataLoader = torch.utils.data.DataLoader(testDataSet, batch_size=batch_size)

min_loss_epoch = 0
min_loss_value = -1
best_model_weights_paths = {}

best_val_loss = float('inf')  # Initialize with a large value
best_epoch = -1
best_model = None

train_losses = []
val_losses = []

criterion = nn.CrossEntropyLoss()
model = CNNmodel()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 1000

writer = SummaryWriter('runs/cnn2'+time.strftime("%Y%m%d-%H%M%S"))

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(trainDataLoader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(trainDataLoader) / batch_size
    train_losses.append(train_loss)
    
    # Validation loop
    model.eval()  
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for images, labels in testDataLoader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
    accuracy = correct_predictions / total_samples
    val_loss = 1-accuracy
    val_losses.append(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch
        best_model = model

    writer.add_scalar('Training Loss', train_loss, epoch)
    writer.add_scalar('Validation Loss', val_loss, epoch)
    
    
    
    


Epoch 1/1000:   0%|          | 0/99 [00:00<?, ?batch/s]

Epoch 1000/1000: 100%|██████████| 99/99 [00:01<00:00, 76.65batch/s]


In [12]:

best_epoch = val_losses.index(min(val_losses))
min_loss_epoch = best_epoch
min_loss_value = f'{min(val_losses):.4f}'
print(f"Min Train Loss: {min(train_losses)} at Epoch {train_losses.index(min(train_losses))}  Min Val Loss: {min_loss_value[criterion]} at Epoch {best_epoch}")



TypeError: string indices must be integers

In [11]:
best_model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in testDataLoader:
        outputs = best_model(images)
        _, predicted = torch.max(outputs, 1)
        
        # Convert to numpy arrays
        predicted_np = predicted.cpu().numpy()
        labels_np = labels.cpu().numpy()

        # Append to the lists
        all_labels.extend(labels_np)
        all_predictions.extend(predicted_np)

# Create confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)

# Plot confusion matrix using seaborn
plt.figure(figsize=(8, 6))
class_names = [str(i) for i in range(len(conf_matrix))]
sns.heatmap(conf_matrix, annot=False, fmt="g", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
